In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator

In [7]:
def correlate(sliding, fixed, sat1Name, sat2Name, key):
    dataRows = []
    numWindows = len(fixed['Time']) - 59

    for j in range(numWindows): # Find the correlation coefficient over 'numWindows' hour-intervals
        sliding_start = (sliding.loc[sliding['Time'] == fixed['Time'][j]]).index[0] # Set the start time of the sliding index where the timestamp of the fixed series begins
        sliding_stop = (sliding.loc[sliding['Time'] == fixed['Time'][j+59]]).index[0] # Set the end of the sliding index an hour after the start of the fixed series

        pMaxes = []
        ratioMaxes =[]
        pOffsets = []
        ratioOffsets = []

        corrStore = [] # Temp storage array for coefficients
        avgStore = [] # Temp storage array for ratios

        fig, ax = plt.subplots(figsize=(8, 5))

        for n in range(31): # Calculate a coefficient over a 0 to 30-min window
            corrStore.append(np.corrcoef(fixed[key][j:j+59], sliding[key][sliding_start-n:sliding_stop-n])[0, 1])  # Append the storage array with the correlation coefficient
            avgStore.append(np.average([(((m+n)*0.5)/abs(m-n)) for m, n in zip(sliding[key][sliding_start - n:sliding_stop - n].to_numpy(), fixed[key][j:j + 59].to_numpy())]))

        ax.scatter(np.arange(0, 31, 1), corrStore, color='C0', label="Corr")
        ax.scatter(np.arange(0, 31, 1), avgStore, color='C1', label="Ratio")

        if all(c < 0 for c in corrStore[1:]):
            pMaxes.append(max(corrStore[1:], key=abs))
            pOffsets.append(corrStore.index(max(corrStore[1:], key=abs)))
            pMaxLineHeight = max(corrStore[1:], key=abs)
            pShift = corrStore.index(max(corrStore[1:], key=abs))

        else:
            pMaxes.append(max(corrStore[1:]))
            pOffsets.append(corrStore.index(max(corrStore[1:])))
            pMaxLineHeight = max(corrStore[1:])
            pShift = corrStore.index(max(corrStore[1:]))


        if avgStore.index(max(avgStore)) == 0:
            avgStore[0] = 0
        ratioMaxes.append(max(avgStore))
        ratioOffsets.append(avgStore.index(max(avgStore)))
        rShift = avgStore.index(max(avgStore))

        ax.vlines(pShift, 0, pMaxLineHeight, linestyle='dashed', color='C0')
        ax.vlines(rShift, 0, max(avgStore, key=abs), linestyle='dashed', color='C1')

        ax.hlines(0, 0, 30, color='black')
        ax.set_ylim(-1, 1)
        ax.set_xlim(0, 30)
        ax.set_title("Peak Corr. Coef./Hourly Ratio")
        ax.set_ylabel("Correlation Coefficient", color='C0')
        ax.set_xlabel("Timeshift (min)")
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        axr = ax.twinx()
        axr.set_ylim(-1, 1)
        axr.set_ylabel("Avg. Ratio of Series ({})".format(key), color='C1')

        if os.path.exists('../metadata/testplots/{}-{}_{}/{}/'.format(sat1Name, sat2Name, fixed['Time'][j].strftime('%Y-%m-%d'), key)):
            plt.savefig('../metadata/testplots/{}-{}_{}/{}/{}-{}_{}.png'.format(sat1Name, sat2Name, fixed['Time'][j].strftime('%Y-%m-%d'), key, sat1Name, sat2Name, fixed['Time'][j].strftime('%H-%M-%S')), dpi=300)
        else:
            os.makedirs('../metadata/testplots/{}-{}_{}/{}/'.format(sat1Name, sat2Name, fixed['Time'][j].strftime('%Y-%m-%d'), key))
            plt.savefig('../metadata/testplots/{}-{}_{}/{}/{}-{}_{}.png'.format(sat1Name, sat2Name, fixed['Time'][j].strftime('%Y-%m-%d'), key, sat1Name, sat2Name, fixed['Time'][j].strftime('%H-%M-%S')), dpi=300)
        plt.close()

        dataRows.append(np.concatenate(([fixed['Time'][j]], [fixed['Time'][j+59]], pMaxes, pOffsets, ratioMaxes, ratioOffsets), axis=None))

    return dataRows #Output the entire chunk of metadata

In [8]:
sat1Directory = '../metadata/Artemis/Artemis_2013-01-17_10-30.csv'
sat1Name = 'Artemis'
sat2Directory = '../metadata/Omni/Omni_2013-01-17_10-30.csv'
sat2Name = 'Omni'

sat1Data = pd.read_csv(sat1Directory, delimiter=',', header=0)
sat1Data['Time'] = pd.to_datetime(sat1Data['Time'], format='%Y-%m-%d %H:%M:%S')
sat2Data = pd.read_csv(sat2Directory, delimiter=',', header=0)
sat2Data['Time'] = pd.to_datetime(sat2Data['Time'], format='%Y-%m-%d %H:%M:%S')

eventMetadata = pd.DataFrame(correlate(sat1Data, sat2Data, sat1Name, sat2Name, 'BX_GSE'), columns=['Start', 'Stop', 'P(Bx)', 'pOffset (Bx)', 'PD(Bx)', 'PDOffset (Bx)'])

eventMetadata.to_csv('../metadata/{}.csv'.format('2013-01-17(test)'))

In [29]:
list = [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10]

if all(l < 0 for l in list):
    print("All less than zero")
else:
    print("I am broken")

All less than zero


In [66]:
list = [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10]
print(list)
list[0] = 0
print(list)
print(min(list))

[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10]
[0, -2, -3, -4, -5, -6, -7, -8, -9, -10]
-10


In [69]:
list = [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10]
print(max(list[1:], key=abs))
print(list.index(max(list[1:], key=abs)))

-10
9
